# Cross Beam Energy Transfer
What is the process for CBET?

## Initialization
### 2D Initialization
$$
\begin{align*}
N_x &= 201 \\
x_{\text{min,max}} &= \pm 5.0\times10^{-4} \text{ cm} \\
dx &= \frac{x_{\text{max}} - x_{\text{min}}}{N_x - 1} 
\end{align*}
$$
z values are identical.

eden: Electron Density, $n_e$ $(N_x \times N_z)$
$$
  \max(0, (0.3 n_c - 0.1 n_c)\frac{x - x_{min}}{x_{max} - x_{min}}+ 0.1 n_c)
$$

dedendx/z: $\frac{\partial n_e}{d (x,z)}$ $(N_x \times N_z)$

edep: Electron deposition, $(N_x + 2 \times N_z + 2)$ Larger to accomodate ray deposition interpolation.

etemp: Electron temperature $(N_x \times N_z)$
$$
  T = 100 \text{ eV? K?}
$$

machnum: Mach number $(N_x \times N_z)$
$$
  \max(0, ((-0.4) - (-2.4))\frac{x - x_{min}}{x_{max} - x_{min}} + (-2.4))
$$

Wavelengths/Frequencies
$$
\begin{align*}
  \lambda &= \frac{1}{3} 1.053\times10^{-4} \text{ cm} = 351 \text{ nm} \\
  f &= \frac{c}{\lambda}\\
  \omega &= 2\pi f \\
  n_{c} &= 10^{-6} \left( \frac{\omega^2 m_e \epsilon_0}{e^2} \right) \\
  \omega_{pe} &= \left[ \frac{10^{-6} n_e e^2}{m_e \epsilon_0} \right]^{1/2}
\end{align*}
$$

beam_min/max_z: $\pm 3 \times 10^{-4}$ cm

phase_x: Beam spatial dependence, $\phi_x$

sigma: Beam width, $\sigma = 1.7 \times 10^{-4}$

pow_x: Beam power, $I(x) = \exp\left(-\left( \left(\frac{\phi_x}{\sigma}\right)^{2} \right)^{4/2}\right)$

intensity: $2 \times 10^{15}$ W/cm^2

offset: $0.5 \times 10^{-4}$, not sure what this is for

x0: Array $N_{\text{rays}}$, `xmin - (dt / cfl * c * 0.5)`

z0: Array $N_{\text{rays}}$, `linspace(beam_min_z, beam_max_z, nrays) + offset - (dz / 2) - (dt / cfl * c * 0.5)`

kx0/kz0: unnormalized k vectors, `1.0, -0.1`

finalts: $N_{\text{rays}} \times N_{\text{beams}}$

uray_mult: `intensity * cfl / rays_per_zone`, not sure what this is for

### Beam/Ray Initialization
The following loop is the same for any beam. `x0`, `z0`, `kx0`, and `kz0` are redefined for each beam.

```py
# Does this get used anywhere?
uray_b1: uray_mult * interp(y=pow_x, x=phase_x + offset, z0)

for n in nrays:
  # Initial power weighting
  uray(1) = uray_mult * interp(pow_x, phase_x + offset, z0(n))
  
  # Total energy injected
  injected += uray(1)

  # Launch ray n
  dummy = launch_ray_XZ(x0(n), z0(n), kx0(n), kz0(n))

  # finalt is a global variable set in launch_ray_XZ
  finalts(n, beam) = finalt

  # rayx & rayz are global variables set in launch_ray_XZ
  mysaved_x(1:finalt, n, beam) = rayx
  mysaved_z(1:finalt, n, beam) = rayz

```
uray_b1: ???

uray: ???

finalts: Final ray times?

mysaved_x: ???

mysaved_z: ???

### Beam Intersection Finding

numstored: `5 * rays_per_zone`

marked: Array $N_x \times N_z \times numstored \times N_{beams}$
  - I'm not sure where this gets filled. It starts off all zeros, and then gets accessed. 
  
present: Array $N_x \times N_z \times N_{beams}$

intersections: Array $N_x \times N_z$

```py
# Iterate over all cells
for xx in range(1, nx + 1):
  for zz in range(1, nz + 1):
    # Loop over stored rays
    for ss in range(numstored + 1):
      if (marked(xx, zz, ss, 1) == 0):
        # No ray found from beam 1
        break
      else:
        # Ray present from beam 1
        iray1 = marked(xx, zz, ss, 1)
        # Iterate over all rays in beam 2
        for sss in range(numstored + 1):
          if (marked(xx, zz, sss, 2) == 0):
            # No ray found from beam 2
            break
          else:
            # Ray present from beam 2
            intersections(xx, zz) += 1.0
```

## CBET Interactions, Gain Coefficients, Solution Iteration

estat: electron charge in statcoloumbs

mach: $-\sqrt{2}$, mach number for max resonance

Z: 3.1, ionization state

m_i: $10230 (10^3 m_e)$, ion mass in grams

Te: $2\times10^3 (11604.5052)$, electron temp in K

Ti: $1\times10^3 (11604.5052)$, ion temp in K

iaw: 0.2, ion-acoustic wave energy damping rate 

constant1: $\frac{e^2_{\text{stat}}}{4\times10^3 m_e c \omega K_b, T_e (1 + \frac{3 T_i}{Z T_e})}$,
  - Used for `gain1` and `gain2`

cs: $10^2 \left[ e_c (Z T_{e, eV} + 3 Ti_{eV}) / m_{i, kg} \right]^{1/2}$, acoustive wave speed

u_flow: `machnum * cs`, plasma flow velocity

boxes: Array $N_{beams} \times N_{rays} \times N_{crossings}$
  - Initialized to zero but never updated

crossesx/z: $N_{beams} \times N_{rays} \times N_{crossings}$
  - Initialized to zero but never updated.

dkx: `crossesx(:, :, 2:0) - crossesx(:, :, 1:-1)`

dky: `crossesz(:, :, 2:0) - crossesz(:, :, 1:-1)`

dkmag: $\sqrt{{dkx}^2 + {dkz}^2}$

W1: $\frac{\sqrt{1 - \frac{n_e}{n_{crit}}}}{\text{rays\_per\_zone}}$

W2: $\frac{\sqrt{1 - \frac{n_e}{n_{crit}}}}{\text{rays\_per\_zone}}$

```py
for bb in range(nbeams):
  for rr1 in range(nrays + 1):
    for cc1 in range(ncrossings + 1):
      ix = boxes(bb, rr1, cc1, 1)
      iz = boxes(bb, rr1, cc1, 2)

      if (intersections(ix, iz) != 0):
        nonzeros1 = where(marked(ix, iz, , 1))
        numrays1 = numberof(nonzeros1)

        nonzeros2 = where(marked(ix, iz, , 2))
        numrays2 = numberof(nonzeros2)

        marker1 = marked(ix, iz, , 1)(nonzeros1)
        marker2 = marked(ix, iz, , 2)(nonzeros2)

        rr2 = marker2
        cc2 = marker2

        for rrr in range(numrays1 + 1):
          if (marker1(rrr) == rr1):
            ray1num = rrr
            break

        for n2 in range(numrays2 + 1):
          for ccc in range(ncrossing + 1):
            ix2 = boxes(bb + 1, rr2(n2), ccc, 1)
            iz2 = boxes(bb + 1, rr2(n2), ccc, 2)
            if (ix == ix2 and iz == iz2):
              cc2(n2) = ccc
              break
        
        n2limit = min(present(ix, iz, 1), numrays2)

        for n2 in range(n2limit):
          ne = eden(ix, iz)
          epsilon = 1 - ne/ncrit
          # magnitude of wave vector
          kmag = (omega / c) * sqrt(epsilon)

          kx1 = kmag * (dkx(bb, rr1, cc1) / (dkmag(bb, rr1, cc1) + 1E-10))
          kx2 = kmag * (dkx(bb + 1, rr2(n2), cc2(n2)) / (dkmag(bb + 1, rr2(n2), cc2(n2)) + 1E-10))

          kz1 = kmag * (dkz(bb, rr1, cc1) / (dkmag(bb, rr1, cc1) + 1E-10))
          kz2 = kmag * (dkz(bb + 1, rr2(n2), cc2(n2)) / (dkmag(bb + 1, rr2(n2), cc2(n2)) + 1E-10))

          kiaw = sqrt((kx2 - kx1)**2 + (kz2 - kz1)**2)

          # acoustic frequency
          ws = kiaw * cs

          # laser frequency difference, initially zero
          omega1 = omega
          omega2 = omega

          eta = ((omega2 - omega1) - (kx2 - kx1) * u_flow(ix, iz)) / (ws + 1.0E-10)

          # Initial electric fields
          efield1 = sqrt(8 * pi * 1E7 * i_b1(ix, iz) / c)
          efield2 = sqrt(8 * pi * 1E7 * i_b2(ix, iz) / c)

          # From Russ Follets paper
          P = (iaw**2 * eta) / ((eta**2 - 1)**2 + (iaw**2 * eta**2))

          gain1 = constant1 * efield2**2 * (ne / ncrit) * (1 / iaw) * P
          gain2 = constant1 * efield1**2 * (ne / ncrit) * (1 / iaw) * P

          # new energy of crossing (probe) ray (beam 2)
          if (dkmag(bb + 1, rr2(n2), cc2(n2)) >= dx):
            W2_new(ix, iz) = W2(ix, iz) * exp(-W1(ix, iz) * dkmag(bb + 1, rr2(n2), cc2(n2)) * gain2 / sqrt(epsilon))
            W2_storage(ix, iz, n2) = W2_new(ix, iz)

            # New enegy of primary (pump) ray (beam 1)
            # Use W1_new formula
            W1_new(ix, iz) = W1(ix, iz) * exp(W2(ix, iz) * dkmag(bb, rr1, cc1) * gain2 / sqrt(epsilon))

            # Enforce energy conservation
            # W1_new(ix, iz) = W1(ix, iz) - (W2_new(ix, iz) - W2(ix, iz))

            W1_storage(ix, iz, ray1num) = W1_new(ix, iz)
            # W1(ix, iz) = W1_new(ix, iz)
```

## Update Intensities

```py
i_b1_new = i_b1
i_b2_new = i_b2

for bb in range(nbeams):
  for rr1 in range(nrays + 1):
    for cc1 in range(ncrossings):
      ix = boxes(bb, rr1, cc1, 1)
      iz = boxes(bb, rr1, cc1, 2)

      if (intersections(ix, iz) != 0):
        nonzeros1 = where(marked(ix, iz, , 1))
        numrays1 = numberof(nonzeros1)

        nonzeros2 = where(marked(ix, iz, , 2))
        numrays2 = numberof(nonzeros2)

        marker1 = marked(ix, iz, , 1)(nonzeros1)
        marker2 = marked(ix, iz, , 2)(nonzeros2)

        rr2 = marker2
        cc1 = marker2

        for rrr in range(numrays1):
          if (marker1(rrr) == rrr):
            ray1num = rrr
            break
        
        for n2 in range(numrays2):
          for ccc in range(ncrossings):
            ix2 = boxes(bb + 1, rr2(n2), ccc, 1)
            iz2 = boxes(bb + 1, rr2(n2), ccc, 2)
            if (ix == ix2 and iz == iz2):
              cc2(n2) = ccc
              break

        fractional_change_1 = -1 * (1 - (W1_new(ix, iz) / W1_init(ix, iz))) * i_b1(ix, iz)
        fractional_change_2 = -1 * (1 - (W2_new(ix, iz) / W2_init(ix, iz))) * i_b2(ix, iz)

        i_b1_new(ix, iz) += fractional_change_1
        i_b2_new(ix, iz) += fractional_change_2

        x_prev_1 = x(ix, iz)
        z_prev_1 = z(ix, iz)

        x_prev_2 = x(ix, iz)
        z_prev_2 = z(ix, iz)

        # Find and Inc/Dec fractional change of rest of the beam 1 ray
        for ccc in range(cc1 + 1, ncrossings):
          ix_next_1 = boxes(1, rr1, ccc, 1)
          iz_next_1 = boxes(1, rr1, ccc, 2)

          x_curr_1 = x(ix_next_1, iz_next_1)
          z_curr_1 = z(ix_next_1, iz_next_1)

          if (ix_next_1 == 0 or iz_next_1 == 0):
            break
          else:
            # Avoid double deposition if (x,z) loc doesn't change
            # with incremented crossing number
            if (x_curr_1 != x_prev_1 or z_curr_1 != z_prev_1):
              i_b1_new(ix_next_1, iz_next_1) += fractional_change_1 * (present(ix, iz, 1) / present(ix_next_1, iz_next_1, 1))
            
            x_prev_1 = x_curr_1
            z_prev_1 = z_curr_1

        # in case numrays2 is less than ray1num of the rays from beam 1 in the box
        n2 = min(ray1num, numrays2)

        # Inc/Dec the fractional change for the rest of beam 2 ray
        for ccc in range(cc2(n2) + 1, ncrossings):
          ix_next_2 = boxes(2, rr2(n2), ccc, 1)
          iz_next_2 = boxes(2, rr2(n2), ccc, 2)

          x_curr_2 = x(ix_next_2, iz_next_2)
          z_curr_2 = z(ix_next_2, iz_next_2)

          if (ix_next_2 == 0 or iz_next_2 == 0):
            break
          else:
            # avoid double deposition if (x,z) doesn't change with incremented crossing number
            if (x_curr_2 != x_prev_2 or z_curr_2 != z_prev_2):
              i_b2_new(ix_next_2, iz_next_2) += fractional_change_2 * (present(ix, iz, 1) / present(ix_next_2, iz_next_2, 2))
            x_prev_2 = x_curr_2
            z_prev_2 = z_curr_2
```

## Launching Rays

```py
def launch_ray_xz(x_init, z_init, kx_init, kz_init):
  global rayx, rayz, finalt, amp_norm, time

  # Find logical indices that corresponds to inputs
  for xx in range(nx):
    if (myx(1) - x(xx, 1) <= (0.5 + 1E-10) * dx and myx(1) - x(xx, 1) >= -(0.5 + 1E-10) * dx):
      thisx_0 = xx
      thisx_00 = xx
      break

  for zz in range(nz):
      if (myz(1) - z(zz, 1) <= (0.5 + 1E-10) * dz and myz(1) - z(zz, 1) >= -(0.5 + 1E-10) * dz):
      thisz_0 = zz
      thisz_00 = zz
      break

  # Calculate total k from the dispersion relation
  # taking into accoutn local plasma frequency where ray starts
  k = sqrt((omega**2 - wpe(thisx_0, thisz_)**2) / c**2)

  knorm = sqrt(kx_init**2 + kz_init**2)

  mykx(1) = (kx_init / knorm) * k
  mykz(1) = (kz_init / knorm) * k
  myvx(1) = c**2 * mykx(1) / omega
  myvz(1) = c**2 * mykz(1) / omega

  numcrossing = 1

  for tt in range(1, nt):
    # Update group velocities using Grad(n_e)
    # Update positions with
    # v_x = v_x0 + -c^2/(2*nc) * d[ne(x)]/dx * dt
    # v_z = v_z0 + -c^2/(2*nc) * d[ne(z)]/dz * dt
    # x_x = x_x0 + v_x * dt
    # x_z = x_z0 + v_z * dt	

    myvz(tt) = myvx(tt - 1)  - c**2 / (2 * ncrit) * dedendx(thisx_0, thisz_0) * dt
    myvz(tt) = myvz(tt - 1)  - c**2 / (2 * ncrit) * dedendz(thisx_0, thisz_0) * dt
    myx(tt) = myx(tt - 1) + myvx(tt) * dt
    myz(tt) = myz(tt - 1) + myvz(tt) * dt

    # Update the index and track intersections and boxes
    for xx in range(nx):
      if (myx(tt) - x(xx, 1) <= (0.5 + 1E-10) * dx and myx(tt) - x(xx, 1) >= -(0.5 + 1E-10) * dx):
        thisx = xx
        break
    for zz in range(nz):
      if (myz(tt) - z(zz, 1) <= (0.5 + 1E-10) * dz and myz(tt) - z(zz, 1) >= -(0.5 + 1E-10) * dz):
        thisz = zz
        break

    for xx in range(nx):
      currx = x(xx, 1) - dx / 2
      if (myx(tt) > currx and myx(tt - 1) <= currx) or (myx(tt) < currx and myx(tt - 1) >= currx):
        # interpolate currx into current x and z segment
        # to get z point of intersection
        crossx = interp([myz(tt - 1), myz(tt)], [myx(tt - 1), myx(tt)], currx)
        if (abs(crossx - lastz) > 1E-20):
          ints(beam, raynum, numcrossing) = uray(tt)
          crossesx(beam, raynum, numcrossing) = currx
          crossesz(beam, raynum, numcrossing) = crossx

          if (myx(tt) <= xmax + dx / 2 and myx(tt) >= xmin - dx / 2):
            boxes(beam, raynum, numcrossing, ) = [thisx, thisz]

          # Keep track of last x-value added
          # to avoid double counting corners
          lastx = currx
          numcrossing += 1
          break
    
    for zz in range(nz):
      currz = z(zz, 1) - dz / 2
      if (myz(tt) > currz and myz(tt - 1) <= currz) or (myz(tt) < currz and myz(tt - 1) >= currz):
        # interpolate currz into current x and z segment
        # to get x point of intersection
        crossx = interp([myx(tt - 1), myx(tt)], [myz(tt - 1), myz(tt)], currz)
        if (abs(crossx - lastz) > 1E-20):
          ints(beam, raynum, numcrossing) = uray(tt)
          crossesx(beam, raynum, numcrossing) = crossz
          crossesz(beam, raynum, numcrossing) = currz

          if (myz(tt) <= zmax + dz / 2 and myz(tt) >= zmin - dz / 2):
            boxes(beam, raynum, numcrossing, ) = [thisx, thisz]

          lastz = currz
          numcrossing += 1
          break
    
    thisx_0 = thisx
    thisz_0 = thisz

    markingx(tt) = thisx
    markingz(tt) = thisz

    if (markingx(tt) != markingx(tt - 1) and markingz(tt) != markingz(tt - 1)):
      if (myvz(tt) < 0.0):
        ztarg = z(thisx, thisz) + dz / 2
      elif (myvz(tt) >= 0.0):
        ztarg = z(thisx, thisz) - dz / 2

      slope = (myz(tt) - myz(tt - 1)) / (myx(tt) - myx(tt - 1) + 1E-10)
      xtarg = myx(tt - 1) + (ztarg + myz(tt - 1)) / slope

      xbounds(tt) = xtarg
      zbounds(tt) = ztarg

      if (myvx(tt) >= 0.0):
        xtarg = x(thisx, thisz) - dx / 2
      elif (myvx(tt) >= 0.0):
        xtarg = x(thisx, thisz) + dx / 2
      
      slope = (myx(tt) - myx(tt - 1)) / (myz(tt) - myz(tt - 1) + 1E-10)
      ztarg = myz(tt - 1) + (xtarg - myx(tt - 1)) / slope

      xbounds_double(tt) = xtarg
      zbounds_double(tt) = ztarg

      for ss in range(numstored):
        if (marked(thisx, thisz, ss, beam) == 0):
          marked(thisx, thisz, ss, beam) = raynum
          present(thisx, thisz, beam) += 1
          break
      
    elif (markingz(tt) != markingz(tt - 1)):
      if (myvz(tt) < 0.0):
        ztarg = z(thisx, thisz) + dz / 2
      elif (myvz(tt) >= 0.0):
        ztarg = z(thisx, thisz) - dz / 2
      
      slope = (myz(tt) - myz(tt - 1)) / (myx(tt) - myx(tt - 1) + 1E-10)
      xtarg = myx(tt - 1) + (ztarg - myz(tt - 1)) / slope

      xbounds(tt) = xtarg
      zbounds(tt) = ztarg

      for ss in range(numstored):
        if (marked(thisx, thisz, ss, beam) == 0):
          marked(thisx, thisz, ss, beam) = raynum
          present(thisx, thisz, beam) += 1
          break
    elif (markingx(tt) != markingx(tt - 1)):
      if (myvx(tt) >= 0.0):
        xtarg = x(thisx, thisz) - dx / 2
      elif (myvx(tt) < 0.0):
        xtarg = x(thisx, thisz) + dx / 2
      
      slope = (myx(tt) - myx(tt - 1)) / (myz(tt) - myz(tt - 1) + 1E-10)
      ztarg = myz(tt - 1) + (xtarg - myx(tt - 1)) / slope

      xbounds(tt) = xtarg
      zbounds(tt) = ztarg

      for ss in range(numstored):
        if (marked(thisx, thisz, ss, beam) == 0):
          marked(thisx, thisz, ss, beam) = raynum
          present(thisx, thisz, beam) += 1
          break
    
    uray(tt) = uray(tt - 1)
    increament = uray(tt)

    xp = (myx(tt) - (x(thisx, thisz) + dx / 2)) / dx
    zp = (myz(tt) - (z(thisx, thisz) + dz / 2)) / dz

    
```

